In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Data 전처리

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [5]:
print(char_dic)

{'i': 0, ' ': 1, 'k': 2, 's': 3, 'b': 4, 'h': 5, 'c': 6, 'o': 7, 'u': 8, 'y': 9, 'd': 10, 'p': 11, 'f': 12, 'w': 13, "'": 14, 'm': 15, 'a': 16, 'n': 17, 'e': 18, 't': 19, ',': 20, 'r': 21, '.': 22, 'g': 23, 'l': 24}


In [6]:
dic_size = len(char_dic)
print(dic_size)

25


In [17]:
# set Hyperparamete
hidden_size = dic_size
sequence_length = 10 # 임의의 num
learning_rate = 0.1

In [8]:
# 임의로 정한 sequence_length 단위로 sample들을 잘라서 data를 만든다
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length] # sequence_length 만큼 cut
    y_str = sentence[i + 1: i + sequence_length + 1] # x에서 1만큼 shift
    print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
print(x_data[0])
print(y_data[0])

[0, 12, 1, 9, 7, 8, 1, 13, 16, 17]
[12, 1, 9, 7, 8, 1, 13, 16, 17, 19]


In [10]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [11]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [12]:
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

### Model 구현하기

In [13]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias = True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [14]:
# 2개의 층 쌓기
net = Net(dic_size, hidden_size, 2)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
outputs = net(X)
print(outputs.shape) # 3차원 Tensor

torch.Size([170, 10, 25])


In [20]:
print(outputs.view(-1, dic_size).shape) # 2차원 Tensor로 전환

torch.Size([1700, 25])


In [21]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [22]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

smimmmiiiiiiiiimiiiiimiaiiiiimiiiiiiimiimiiiiiiiaiiiimiiiiiiiiiiiiiiiiiiimimiiimiiiiiiiiiiiiiiiiiiiiimimiiiiiiiiiiiiiimiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiimiiiiiimiiiiaimiiiiiiiii
                                                                                                                                                                                   
eoeoeeooeo eo to oo to  oeo oeet  to o eo to ooeooo oo eo to to to o eo oeeo  oe o  to  oe o  t eo oo to  eo oo to  oeeo oeto eo to o eo eo to o  t  teeo eo oeoooo oo  t  to t  oe
w. t..  ..sw.stws w. t. ..h w.s .sw.st..s  ,. w.st.s.s  .. w..syw,s w..  ..s  .. t.s ws ,. t.st....s...ts ,.  ,s w. w.s w.s t..h .. t.ft....sw. .... w.s w.sp ..  .. w.st.st.st.s .
nntn wh nssnn  stn  ns n  h s nd th ws  h whtn wh n nn    hs ss n th bh s ts     n  hnn t  s  h th t  hns    nn nn hn sn h ts ns n ts wh nh n th th  h  h nh s hn n    hssth td th 
nun  to nu l  nn  n lulo tu bo no    nululnoln n to ln  h oulth o tm b ln tn  b lnn  nn nn tmlnuloln

totor want to luild a ship, don't drum up people together to collect wood and won't dssign them tosks and work, but rather teach the  to long for the endless immensity of themeeac
totor want to luild a ship, don't arum up people together to collect wood and won't assign them tasks and work, but rather teach the  ta long for the endless immensity of the eeac
totor want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach the  ta long for the endless immensity of the eeac
totor want to build a ship, don't arum up people together to collect wood and don't assign them tasks and dork, but rather teach them to bong for the endless immensity of the eeac
totor want to build a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather teach them to bong for the endless immensity of themeeac
totor want to build a ship, don't arum up people together to collect wood and don't assign them task

p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't dssign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
